In [154]:
#import api keys
from api_keys import *

## Flights API

In [181]:
import requests

airport_icoa = "EDDB"
to_local_time = "2021-10-04T20:00"
from_local_time = "2021-10-05T08:00"

url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{airport_icoa}/{to_local_time}/{from_local_time}"

querystring = {"withLeg":"true","withCancelled":"true","withCodeshared":"true","withCargo":"true","withPrivate":"false","withLocation":"false"}

headers = {
    'x-rapidapi-host': "aerodatabox.p.rapidapi.com",
    'x-rapidapi-key': flight_api_key
    }

response = requests.request("GET", url, headers=headers, params=querystring)

from IPython.display import JSON
JSON(response.json())

<IPython.core.display.JSON object>

In [182]:
### Option 1
arrivals_berlin = response.json()['arrivals']

def get_flight_info(flight_json):
    # terminal
    try: terminal = flight_json['arrival']['terminal']
    except: terminal = None
    # aircraft
    try: aircraft = flight_json['aircraft']['model']
    except: aircraft = None

    return {
        'dep_airport':flight_json['departure']['airport']['name'],
        'sched_arr_loc_time':flight_json['arrival']['scheduledTimeLocal'],
        'terminal':terminal,
        'status':flight_json['status'],
        'aircraft':aircraft,
        'icao_code':airport_icoa
    }


import pandas as pd
# [get_flight_info(flight) for flight in arrivals_berlin]
arrivals_berlin = pd.DataFrame([get_flight_info(flight) for flight in arrivals_berlin])
arrivals_berlin

,dep_airport,sched_arr_loc_time,terminal,status,aircraft,icao_code
0,Olbia,2021-10-04 19:59+02:00,None,Arrived,Airbus A320-100/200,EDDB
1,London,2021-10-04 20:35+02:00,1,Arrived,Airbus A320-200 (Sharklets),EDDB
2,Unknown,2021-10-04 20:14+02:00,None,Arrived,Airbus A320-100/200,EDDB
3,Oslo,2021-10-04 20:05+02:00,1,Arrived,Boeing 737-800,EDDB
4,Athens,2021-10-04 20:33+02:00,None,Arrived,Airbus A320-100/200,EDDB
...,...,...,...,...,...,...
86,Stuttgart,2021-10-05 07:35+02:00,1,Unknown,Airbus A320,EDDB
87,Duesseldorf,2021-10-05 07:35+02:00,1,Unknown,Airbus A320,EDDB
88,Frankfurt-am-Main,2021-10-05 07:55+02:00,1,Arrived,Airbus A321-100/200,EDDB
89,Varna,2021-10-05 07:40+02:00,0,Arrived,Airbus A320-100/200,EDDB


In [183]:
### Option 2
import pandas as pd
arrivals = pd.json_normalize(response.json()['arrivals'])
# next step: select the columns you want to incude on your database
(
arrivals
    .filter(['departure.airport.name','arrival.scheduledTimeLocal',
             'arrival.terminal','status','aircraft.model'])
    .assign(icao_code = airport_icoa)
)

,departure.airport.name,arrival.scheduledTimeLocal,arrival.terminal,status,aircraft.model,icao_code
0,Olbia,2021-10-04 19:59+02:00,NaN,Arrived,Airbus A320-100/200,EDDB
1,London,2021-10-04 20:35+02:00,1,Arrived,Airbus A320-200 (Sharklets),EDDB
2,Unknown,2021-10-04 20:14+02:00,NaN,Arrived,Airbus A320-100/200,EDDB
3,Oslo,2021-10-04 20:05+02:00,1,Arrived,Boeing 737-800,EDDB
4,Athens,2021-10-04 20:33+02:00,NaN,Arrived,Airbus A320-100/200,EDDB
...,...,...,...,...,...,...
86,Stuttgart,2021-10-05 07:35+02:00,1,Unknown,Airbus A320,EDDB
87,Duesseldorf,2021-10-05 07:35+02:00,1,Unknown,Airbus A320,EDDB
88,Frankfurt-am-Main,2021-10-05 07:55+02:00,1,Arrived,Airbus A321-100/200,EDDB
89,Varna,2021-10-05 07:40+02:00,0,Arrived,Airbus A320-100/200,EDDB


## Wheather API

### Connecting to OWM API

In [184]:
city = "Berlin"
country = "DE"
# achieve the same result with the wather api
response = requests.get(f'http://api.openweathermap.org/data/2.5/forecast/?q={city},{country}&appid={OWM_key}&units=metric&lang=en')

from IPython.display import JSON
JSON(response.json())

<IPython.core.display.JSON object>

In [185]:
forecast_api = response.json()['list']
# look for the fields that could ve relevant: 
# better field descriptions https://www.weatherbit.io/api/weather-forecast-5-day

weather_info = []

# datetime, temperature, wind, prob_perc, rain_qty, snow = [], [], [], [], [], []
for forecast_3h in forecast_api: 
    weather_hour = {}
    # datetime utc
    weather_hour['datetime'] = forecast_3h['dt_txt']
    # temperature 
    weather_hour['temperature'] = forecast_3h['main']['temp']
    # wind
    weather_hour['wind'] = forecast_3h['wind']['speed']
    # probability precipitation 
    try: weather_hour['prob_perc'] = float(forecast_3h['pop'])
    except: weather_hour['prob_perc'] = 0
    # rain
    try: weather_hour['rain_qty'] = float(forecast_3h['rain']['3h'])
    except: weather_hour['rain_qty'] = 0
    # wind 
    try: weather_hour['snow'] = float(forecast_3h['snow']['3h'])
    except: weather_hour['snow'] = 0
    weather_hour['municipality_iso_country'] = city + ',' + country
    weather_info.append(weather_hour)
    
weather_data = pd.DataFrame(weather_info)
weather_data.head()

,datetime,temperature,wind,prob_perc,rain_qty,snow,municipality_iso_country
0,2021-12-01 12:00:00,6.38,8.16,1.00,0.73,0.0,"Berlin,DE"
1,2021-12-01 15:00:00,6.71,8.30,0.30,0.41,0.0,"Berlin,DE"
2,2021-12-01 18:00:00,7.25,8.32,0.09,0.00,0.0,"Berlin,DE"
3,2021-12-01 21:00:00,7.32,10.44,0.20,0.13,0.0,"Berlin,DE"
4,2021-12-02 00:00:00,5.11,11.10,0.02,0.00,0.0,"Berlin,DE"


## Population data

In [186]:
import requests
import bs4
from bs4 import BeautifulSoup as bs
import pandas as pd
import unicodedata
import re

# cities = ['Berlin', 'Hamburg', 'Frankfurt','Munich','Stuttgart','Leipzig','Cologne','Dresden','Hannover','Paris', 'Barcelona','Lisbon','Madrid']
cities = ['Berlin','Paris','Amsterdam','Barcelona','Rome','Lisbon','Prague','Vienna','Madrid']

def City_info(soup):
    
    ret_dict = {}
    ret_dict['city'] = soup.h1.get_text()
    
    
    if soup.select_one('.mergedrow:-soup-contains("Mayor")>.infobox-label') != None:
        i = soup.select_one('.mergedrow:-soup-contains("Mayor")>.infobox-label')
        mayor_name_html = i.find_next_sibling()
        mayor_name = unicodedata.normalize('NFKD',mayor_name_html.get_text())
        ret_dict['mayor']  = mayor_name
    
    if soup.select_one('.mergedrow:-soup-contains("City")>.infobox-label') != None:
        j =  soup.select_one('.mergedrow:-soup-contains("City")>.infobox-label')
        area = j.find_next_sibling('td').get_text()
        ret_dict['city_size'] = unicodedata.normalize('NFKD',area)

    if soup.select_one('.mergedtoprow:-soup-contains("Elevation")>.infobox-data') != None:
        k = soup.select_one('.mergedtoprow:-soup-contains("Elevation")>.infobox-data')
        elevation_html = k.get_text()
        ret_dict['elevation'] = unicodedata.normalize('NFKD',elevation_html)
    
    if soup.select_one('.mergedtoprow:-soup-contains("Population")') != None:
        l = soup.select_one('.mergedtoprow:-soup-contains("Population")')
        c_pop = l.findNext('td').get_text()
        ret_dict['city_population'] = c_pop
    
    if soup.select_one('.infobox-label>[title^=Urban]') != None:
        m = soup.select_one('.infobox-label>[title^=Urban]')
        u_pop = m.findNext('td')
        ret_dict['urban_population'] = u_pop.get_text()

    if soup.select_one('.infobox-label>[title^=Metro]') != None:
        n = soup.select_one('.infobox-label>[title^=Metro]')
        m_pop = n.findNext('td')
        ret_dict['metro_population'] = m_pop.get_text()
    
    if soup.select_one('.latitude') != None:
        o = soup.select_one('.latitude')
        ret_dict['lat'] = o.get_text()

    if soup.select_one('.longitude') != None:    
        p = soup.select_one('.longitude')
        ret_dict['long'] = p.get_text()
    
    return ret_dict



list_of_city_info = []
for city in cities:
    url = 'https://en.wikipedia.org/wiki/{}'.format(city)
    web = requests.get(url,'html.parser')
    soup = bs(web.content)
    list_of_city_info.append(City_info(soup))
df_cities = pd.DataFrame(list_of_city_info)
# df_cities = df_cities.set_index('city')
df_cities

,city,mayor,city_size,elevation,city_population,urban_population,metro_population,lat,long
0,Berlin,Michael Müller (SPD),891.7 km2 (344.3 sq mi),34 m (112 ft),"3,664,088","4,473,101","6,144,600",52°31′12″N,13°24′18″E
1,Paris,Anne Hidalgo (PS),NaN,28–131 m (92–430 ft) (avg. 78 m or 256 ft),"2,175,601","10,785,092","13,024,518",48°51′24″N,2°21′08″E
2,Amsterdam,Femke Halsema (GL),NaN,−2 m (−7 ft),"872,680","1,558,755",NaN,52°22′N,4°54′E
3,Barcelona,Ada Colau Ballano[1] (Barcelona en Comú),101.4 km2 (39.2 sq mi),12 m (39 ft),"1,620,343","4,840,000[3]","5,474,482[4]",41°23′N,2°11′E
4,Rome,Strong Mayor–Council,"4,342,212[2]",21 m (69 ft),1st in Italy (3rd in the EU),NaN,Rome Capital,41°53′N,12°30′E
5,Lisbon,Carlos Moedas,NaN,2 m (7 ft),"544,851[1]","2,062,444[1]","2,871,133[1]",38°43′31″N,9°09′00″W
6,Prague,Zdeněk Hřib (Pirates),NaN,NaN,"1,335,084",NaN,"2,709,418[4]",50°5′N,14°25′E
7,Vienna,Michael Ludwig (SPÖ),NaN,"151 (Lobau) – 542 (Hermannskogel) m (495–1,778...",1st in Austria (6th in EU),"1,911,191 (01−01−20)","2,600,000",48°12′N,16°22′E
8,Madrid,José Luis Martínez-Almeida (PP),NaN,"820 m (2,690 ft)","3,223,334","6,345,000 (2,019)[3]","6,791,667 (2,018)[2]",40°25′N,3°43′W


## Airports data

In [187]:
import pandas as pd

airports_cities = (
pd.read_csv('airports.csv')
    .query('type == "large_airport"')
    .filter(['name','latitude_deg','longitude_deg','iso_country','iso_region','municipality','gps_code','iata_code'])
    .rename(columns={'gps_code':'icao_code'})
    .assign(municipality_iso_country = lambda x: x['municipality'] + ',' + x['iso_country'])
)
airports_cities.head()

,name,latitude_deg,longitude_deg,iso_country,iso_region,municipality,icao_code,iata_code,municipality_iso_country
10890,Honiara International Airport,-9.428000,160.054993,SB,SB-CT,Honiara,AGGH,HIR,"Honiara,SB"
12461,Port Moresby Jacksons International Airport,-9.443380,147.220001,PG,PG-NCD,Port Moresby,AYPY,POM,"Port Moresby,PG"
12981,Keflavik International Airport,63.985001,-22.605600,IS,IS-2,Reykjavík,BIKF,KEF,"Reykjavík,IS"
13028,Priština Adem Jashari International Airport,42.572800,21.035801,XK,XK-01,Prishtina,BKPR,PRN,"Prishtina,XK"
17254,Guodu Air Base,36.001741,117.632010,CN,CN-37,"Xintai, Tai'an",NaN,NaN,"Xintai, Tai'an,CN"


## Check the tables

In [188]:
arrivals_berlin

,dep_airport,sched_arr_loc_time,terminal,status,aircraft,icao_code
0,Olbia,2021-10-04 19:59+02:00,None,Arrived,Airbus A320-100/200,EDDB
1,London,2021-10-04 20:35+02:00,1,Arrived,Airbus A320-200 (Sharklets),EDDB
2,Unknown,2021-10-04 20:14+02:00,None,Arrived,Airbus A320-100/200,EDDB
3,Oslo,2021-10-04 20:05+02:00,1,Arrived,Boeing 737-800,EDDB
4,Athens,2021-10-04 20:33+02:00,None,Arrived,Airbus A320-100/200,EDDB
...,...,...,...,...,...,...
86,Stuttgart,2021-10-05 07:35+02:00,1,Unknown,Airbus A320,EDDB
87,Duesseldorf,2021-10-05 07:35+02:00,1,Unknown,Airbus A320,EDDB
88,Frankfurt-am-Main,2021-10-05 07:55+02:00,1,Arrived,Airbus A321-100/200,EDDB
89,Varna,2021-10-05 07:40+02:00,0,Arrived,Airbus A320-100/200,EDDB


In [189]:
arrivals_berlin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91 entries, 0 to 90
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   dep_airport         91 non-null     object
 1   sched_arr_loc_time  91 non-null     object
 2   terminal            42 non-null     object
 3   status              91 non-null     object
 4   aircraft            89 non-null     object
 5   icao_code           91 non-null     object
dtypes: object(6)
memory usage: 4.4+ KB


In [190]:
weather_data

,datetime,temperature,wind,prob_perc,rain_qty,snow,municipality_iso_country
0,2021-12-01 12:00:00,6.38,8.16,1.00,0.73,0.00,"Berlin,DE"
1,2021-12-01 15:00:00,6.71,8.30,0.30,0.41,0.00,"Berlin,DE"
2,2021-12-01 18:00:00,7.25,8.32,0.09,0.00,0.00,"Berlin,DE"
3,2021-12-01 21:00:00,7.32,10.44,0.20,0.13,0.00,"Berlin,DE"
4,2021-12-02 00:00:00,5.11,11.10,0.02,0.00,0.00,"Berlin,DE"
5,2021-12-02 03:00:00,3.53,9.09,0.00,0.00,0.00,"Berlin,DE"
6,2021-12-02 06:00:00,1.91,7.27,0.00,0.00,0.00,"Berlin,DE"
7,2021-12-02 09:00:00,2.07,6.27,0.00,0.00,0.00,"Berlin,DE"
8,2021-12-02 12:00:00,3.24,4.72,0.00,0.00,0.00,"Berlin,DE"
9,2021-12-02 15:00:00,1.69,2.69,0.00,0.00,0.00,"Berlin,DE"


In [191]:
weather_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 7 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   datetime                  40 non-null     object 
 1   temperature               40 non-null     float64
 2   wind                      40 non-null     float64
 3   prob_perc                 40 non-null     float64
 4   rain_qty                  40 non-null     float64
 5   snow                      40 non-null     float64
 6   municipality_iso_country  40 non-null     object 
dtypes: float64(5), object(2)
memory usage: 2.3+ KB


In [192]:
airports_cities

,name,latitude_deg,longitude_deg,iso_country,iso_region,municipality,icao_code,iata_code,municipality_iso_country
10890,Honiara International Airport,-9.428000,160.054993,SB,SB-CT,Honiara,AGGH,HIR,"Honiara,SB"
12461,Port Moresby Jacksons International Airport,-9.443380,147.220001,PG,PG-NCD,Port Moresby,AYPY,POM,"Port Moresby,PG"
12981,Keflavik International Airport,63.985001,-22.605600,IS,IS-2,Reykjavík,BIKF,KEF,"Reykjavík,IS"
13028,Priština Adem Jashari International Airport,42.572800,21.035801,XK,XK-01,Prishtina,BKPR,PRN,"Prishtina,XK"
17254,Guodu Air Base,36.001741,117.632010,CN,CN-37,"Xintai, Tai'an",NaN,NaN,"Xintai, Tai'an,CN"
...,...,...,...,...,...,...,...,...,...
68223,Chengdu Shuangliu International Airport,30.578501,103.946999,CN,CN-51,Chengdu,ZUUU,CTU,"Chengdu,CN"
68295,Ürümqi Diwopu International Airport,43.907101,87.474197,CN,CN-65,Ürümqi,ZWWW,URC,"Ürümqi,CN"
68306,Harbin Taiping International Airport,45.623402,126.250000,CN,CN-23,Harbin,ZYHB,HRB,"Harbin,CN"
68321,Dalian Zhoushuizi International Airport,38.965698,121.539001,CN,CN-21,"Ganjingzi, Dalian",ZYTL,DLC,"Ganjingzi, Dalian,CN"


In [193]:
airports_cities.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 624 entries, 10890 to 68323
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   name                      624 non-null    object 
 1   latitude_deg              624 non-null    float64
 2   longitude_deg             624 non-null    float64
 3   iso_country               623 non-null    object 
 4   iso_region                624 non-null    object 
 5   municipality              619 non-null    object 
 6   icao_code                 619 non-null    object 
 7   iata_code                 617 non-null    object 
 8   municipality_iso_country  618 non-null    object 
dtypes: float64(2), object(7)
memory usage: 48.8+ KB


In [194]:
cities = airports_cities.filter(['municipality','iso_country','municipality_iso_country']).drop_duplicates()
cities.head()

,municipality,iso_country,municipality_iso_country
10890,Honiara,SB,"Honiara,SB"
12461,Port Moresby,PG,"Port Moresby,PG"
12981,Reykjavík,IS,"Reykjavík,IS"
13028,Prishtina,XK,"Prishtina,XK"
17254,"Xintai, Tai'an",CN,"Xintai, Tai'an,CN"


In [195]:
airports_cities.merge(arrivals_berlin, on='icao_code', how='inner').merge(weather_data, on='municipality_iso_country', how='inner').head()

,name,latitude_deg,longitude_deg,iso_country,iso_region,municipality,icao_code,iata_code,municipality_iso_country,dep_airport,sched_arr_loc_time,terminal,status,aircraft,datetime,temperature,wind,prob_perc,rain_qty,snow
0,Berlin Brandenburg Airport,52.351389,13.493889,DE,DE-BR,Berlin,EDDB,BER,"Berlin,DE",Olbia,2021-10-04 19:59+02:00,None,Arrived,Airbus A320-100/200,2021-12-01 12:00:00,6.38,8.16,1.00,0.73,0.0
1,Berlin Brandenburg Airport,52.351389,13.493889,DE,DE-BR,Berlin,EDDB,BER,"Berlin,DE",Olbia,2021-10-04 19:59+02:00,None,Arrived,Airbus A320-100/200,2021-12-01 15:00:00,6.71,8.30,0.30,0.41,0.0
2,Berlin Brandenburg Airport,52.351389,13.493889,DE,DE-BR,Berlin,EDDB,BER,"Berlin,DE",Olbia,2021-10-04 19:59+02:00,None,Arrived,Airbus A320-100/200,2021-12-01 18:00:00,7.25,8.32,0.09,0.00,0.0
3,Berlin Brandenburg Airport,52.351389,13.493889,DE,DE-BR,Berlin,EDDB,BER,"Berlin,DE",Olbia,2021-10-04 19:59+02:00,None,Arrived,Airbus A320-100/200,2021-12-01 21:00:00,7.32,10.44,0.20,0.13,0.0
4,Berlin Brandenburg Airport,52.351389,13.493889,DE,DE-BR,Berlin,EDDB,BER,"Berlin,DE",Olbia,2021-10-04 19:59+02:00,None,Arrived,Airbus A320-100/200,2021-12-02 00:00:00,5.11,11.10,0.02,0.00,0.0


## Update data into database

### `sqlalchemy`

#### Establish the connection

In [196]:
!pip install pymysql

In [197]:
!pip install sqlalchemy

In [198]:
#import awswrangler as wr
import pandas as pd
import sqlalchemy

In [199]:
#schema="gans"
#host="127.0.0.1"
#user="root"
#password="Maltepe1124"
#port=3306
#con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [200]:
schema="gang"
host="database-wbs.ceefmumd0igz.eu-central-1.rds.amazonaws.com"
user="admin"
password="Maltepe1124"
port=3306
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

#### Update the tables

In [201]:
cities

,municipality,iso_country,municipality_iso_country
10890,Honiara,SB,"Honiara,SB"
12461,Port Moresby,PG,"Port Moresby,PG"
12981,Reykjavík,IS,"Reykjavík,IS"
13028,Prishtina,XK,"Prishtina,XK"
17254,"Xintai, Tai'an",CN,"Xintai, Tai'an,CN"
...,...,...,...
68223,Chengdu,CN,"Chengdu,CN"
68295,Ürümqi,CN,"Ürümqi,CN"
68306,Harbin,CN,"Harbin,CN"
68321,"Ganjingzi, Dalian",CN,"Ganjingzi, Dalian,CN"


In [204]:
#cities.isna().sum()
#cities.dropna().to_sql('cities', if_exists='append', con=con, index=False)

airports_cities.dropna().to_sql('airports_cities2', if_exists='append', con=con, index=False)

In [205]:
weather_data.assign(datetime = lambda x: pd.to_datetime(x['datetime'])).to_sql('weather', if_exists='append', con=con, index=False)

In [206]:
import numpy as np
(
arrivals_berlin
    .replace({np.nan},'unknown')
    .assign(sched_arr_loc_time = lambda x: pd.to_datetime(x['sched_arr_loc_time']))
    .to_sql('arrivals', if_exists='append', con=con, index=False))

#### Run sql queries in our python session

In [179]:
pd.read_sql(
    sql = """
        select * from arrivals
        where status = "Arrived"
    """,
    con = con
)

,arrival_id,dep_airport,sched_arr_loc_time,terminal,status,aircraft,icao_code
0,547,Athens,2021-10-04 20:33:00,unknown,Arrived,Airbus A320-100/200,EDDB
1,548,Souda,2021-10-04 20:07:00,unknown,Arrived,Airbus A320-100/200,EDDB
2,549,Oslo,2021-10-04 20:05:00,1,Arrived,Boeing 737-800,EDDB
3,551,Palma De Mallorca,2021-10-04 20:25:00,unknown,Arrived,Boeing 737-800 (winglets),EDDB
4,553,Luxembourg,2021-10-04 20:10:00,1,Arrived,De Havilland Canada DHC-8-400 Dash 8Q,EDDB
...,...,...,...,...,...,...,...
677,1540,Stuttgart,2021-10-05 07:03:00,unknown,Arrived,Airbus A320-200 (Sharklets),EDDB
678,1541,Bâle/Mulhouse,2021-10-05 07:19:00,unknown,Arrived,Airbus A319,EDDB
679,1545,Frankfurt-am-Main,2021-10-05 07:55:00,1,Arrived,Airbus A321-100/200,EDDB
680,1546,Varna,2021-10-05 07:40:00,0,Arrived,Airbus A320-100/200,EDDB


### `pymysql`

In [180]:
import pymysql
import awswrangler as wr

con = pymysql.connect(
    database="gang",
    host="127.0.0.1",
    user="root",
    password="Maltepe1124",
    port=3306
)

wr.mysql.to_sql(
    df_cities, 
    con,
    schema="gang",
    table="new_table"
)
if con.open: con.close()

ModuleNotFoundError: No module named 'awswrangler'